In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.insert(1, "../python/functions")
sys.path.insert(2, "../python/architecture")

In [3]:
from data_prep_functions import wine_prep
from plotting_functions import nn_plot_epoch_mse
from enkf_functions import enkf_regressor
from saving_functions import load_objects
from model_functions import nn_load
import reproducible
import no_gpu
from sklearn.metrics import mean_squared_error
import numpy as np

In [4]:
X_train, X_test, y_train, y_test = wine_prep()

In [6]:
layers = 4
neurons = [32, 32, 16, 1]

In [ ]:
import time

In [7]:
setting_dict = {"particles": 1,
                "epochs": 20,
                "batch_size": 200,    # len(X_train)
                "h_0": 2,
                "epsilon": 0.5,
                "randomization": False,
                "shuffle": True,
                "early_stopping": False,
                "early_stopping_diff": 0.001
               }

In [ ]:
particle_list = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1500, 2000]

In [9]:
for p in particle_list:
    setting_dict["particles"] = p
    start_time = time.time()
    mean_model, mean_model_train_mse, mean_model_test_mse = enkf_regressor(X_train,
                                                                           X_test,
                                                                           y_train,
                                                                           y_test,
                                                                           layers,
                                                                           neurons,
                                                                           setting_dict,
                                                                           save_all = True,
                                                                           file_var = "../objects/wine/enkf_wine_full_E{}_B{}_P{}_H{}.pckl".format(setting_dict["epochs"], setting_dict["batch_size"], setting_dict["particles"], setting_dict["h_0"]),
                                                                           file_model = "../models/wine/enkf_wine_full_E{}_B{}_P{}_H{}.h5".format(setting_dict["epochs"], setting_dict["batch_size"], setting_dict["particles"], setting_dict["h_0"]),
                                                                           verbose = 1)
    end_time = time.time()
    print("Calculation time for {} particles: {} minutes.".format(p, (end_time - start_time) / 60))
    nn_plot_epoch_mse(mean_model_train_mse,
                      mean_model_test_mse,
                      mse_mean = None,
                      savefig = True,
                      file = "../img/wine/enkf_wine_full_E{}_B{}_P{}_H{}.png".format(setting_dict["epochs"], setting_dict["batch_size"], setting_dict["particles"], setting_dict["h_0"]))

Epoch 1. Training MSE: 0.572, Test MSE: 0.698.
Epoch 2. Training MSE: 0.554, Test MSE: 0.678.
Epoch 3. Training MSE: 0.55, Test MSE: 0.674.
Epoch 4. Training MSE: 0.548, Test MSE: 0.672.
Epoch 5. Training MSE: 0.547, Test MSE: 0.67.
Epoch 6. Training MSE: 0.546, Test MSE: 0.669.
Epoch 7. Training MSE: 0.545, Test MSE: 0.668.
Epoch 8. Training MSE: 0.544, Test MSE: 0.667.
Epoch 9. Training MSE: 0.543, Test MSE: 0.666.
Epoch 10. Training MSE: 0.543, Test MSE: 0.666.


In [12]:
obj_dict = load_objects("../objects/wine/enkf_wine_full_E{}_B{}_P{}_H{}.pckl".format(setting_dict["epochs"], setting_dict["batch_size"], setting_dict["particles"], setting_dict["h_0"]))

In [13]:
final_model = nn_load("../models/wine/enkf_wine_full_E{}_B{}_P{}_H{}.h5".format(setting_dict["epochs"], setting_dict["batch_size"], setting_dict["particles"], setting_dict["h_0"]))